In [1]:
import argparse
import numpy as np
import random
import argparse
import os
import time
import math
import sys
import json
import csv
import pandas as pd

import torch
from torch.autograd import Variable

from models import load_models_autoencoder, generate
from utils import to_gpu, Corpus, batchify, train_ngram_lm, get_ppl

In [2]:
load_path='output/output_autoencoder1109'

In [2]:
load_path='output/out1102_token'

In [4]:
model_args, idx2word, autoencoder, word2idx= load_models_autoencoder(load_path)

Loading models fromoutput/output_autoencoder1109


In [5]:
vocab2=idx2word
vocab2

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<oov>',
 4: '',
 5: '.',
 6: '1',
 7: '100%가',
 8: '100%보리로만',
 9: '100%보리맥주이고',
 10: '10캔',
 11: '15년',
 12: '15년쯤돼는대',
 13: '1로',
 14: '1위를',
 15: '1차로',
 16: '2.',
 17: '2002년',
 18: '20대',
 19: '20대는',
 20: '20대에',
 21: '20세',
 22: '23',
 23: '2ne1의',
 24: '2pm나오기도',
 25: '2pm이',
 26: '2pm팬이라서요',
 27: '2병마실것을',
 28: '2위를',
 29: '2차로',
 30: '30대인',
 31: '33%낮다고',
 32: '34병까지도',
 33: '3가지만',
 34: '3개월간',
 35: '3년간',
 36: '3분의',
 37: '50%이상',
 38: '500cc',
 39: '500년',
 40: 'Asahi',
 41: 'BI가',
 42: 'CASS라는',
 43: 'CF',
 44: 'CF가',
 45: 'CF가벼운맛.',
 46: 'CF광고가',
 47: 'CF를',
 48: 'CF모델이',
 49: 'CF에',
 50: 'CF에서',
 51: 'CF의',
 52: 'CL',
 53: 'CL양도',
 54: 'Cass',
 55: 'Cass라는',
 56: 'Cf의',
 57: 'GOOD.',
 58: 'Guiness',
 59: 'Hite',
 60: 'KGB',
 61: 'KGB가',
 62: 'KGB같은',
 63: 'KGB는',
 64: 'KGB레몬을',
 65: 'KGB를',
 66: 'KGB맥주를',
 67: 'KGB의',
 68: 'Light는',
 69: 'MAX라고',
 70: 'MAX만',
 71: 'Max',
 72: 'OB가',
 73: 'OB골든라거',
 74: 'OB라고',
 75: 'OB라는',
 7

In [6]:
word2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 '<oov>': 3,
 '': 4,
 '.': 5,
 '1': 6,
 '100%가': 7,
 '100%보리로만': 8,
 '100%보리맥주이고': 9,
 '10캔': 10,
 '15년': 11,
 '15년쯤돼는대': 12,
 '1로': 13,
 '1위를': 14,
 '1차로': 15,
 '2.': 16,
 '2002년': 17,
 '20대': 18,
 '20대는': 19,
 '20대에': 20,
 '20세': 21,
 '23': 22,
 '2ne1의': 23,
 '2pm나오기도': 24,
 '2pm이': 25,
 '2pm팬이라서요': 26,
 '2병마실것을': 27,
 '2위를': 28,
 '2차로': 29,
 '30대인': 30,
 '33%낮다고': 31,
 '34병까지도': 32,
 '3가지만': 33,
 '3개월간': 34,
 '3년간': 35,
 '3분의': 36,
 '50%이상': 37,
 '500cc': 38,
 '500년': 39,
 'Asahi': 40,
 'BI가': 41,
 'CASS라는': 42,
 'CF': 43,
 'CF가': 44,
 'CF가벼운맛.': 45,
 'CF광고가': 46,
 'CF를': 47,
 'CF모델이': 48,
 'CF에': 49,
 'CF에서': 50,
 'CF의': 51,
 'CL': 52,
 'CL양도': 53,
 'Cass': 54,
 'Cass라는': 55,
 'Cf의': 56,
 'GOOD.': 57,
 'Guiness': 58,
 'Hite': 59,
 'KGB': 60,
 'KGB가': 61,
 'KGB같은': 62,
 'KGB는': 63,
 'KGB레몬을': 64,
 'KGB를': 65,
 'KGB맥주를': 66,
 'KGB의': 67,
 'Light는': 68,
 'MAX라고': 69,
 'MAX만': 70,
 'Max': 71,
 'OB가': 72,
 'OB골든라거': 73,
 'OB라고': 74,
 'OB라는': 75,
 '

In [7]:
model_args

{'N': 5,
 'arch_d': '300-300',
 'arch_g': '300-300',
 'batch_size': 5,
 'beta1': 0.9,
 'clip': 1,
 'cuda': True,
 'data_path': 'data_korean/rawdata',
 'dropout': 0.0,
 'emsize': 300,
 'enc_grad_norm': True,
 'epochs': 49,
 'gan_clamp': 0.01,
 'gan_toenc': -0.01,
 'hidden_init': False,
 'kenlm_path': '../Data/kenlm',
 'log_interval': 200,
 'lowercase': False,
 'lr_ae': 1,
 'lr_gan_d': 1e-05,
 'lr_gan_g': 5e-05,
 'maxlen': 30,
 'min_epochs': 50,
 'nhidden': 300,
 'niters_ae': 1,
 'niters_gan_d': 5,
 'niters_gan_g': 1,
 'niters_gan_schedule': '2-4-6',
 'nlayers': 1,
 'no_earlystopping': False,
 'noise_anneal': 0.995,
 'noise_radius': 0.2,
 'ntokens': 10169,
 'outf': 'output_autoencoder1109',
 'patience': 5,
 'sample': False,
 'seed': 1111,
 'temp': 1,
 'vocab_size': 11000,
 'z_size': 100}

# ---------전처리

In [266]:
import re

In [254]:
file = 'codedata/rawinput.csv'
rawdata = pd.read_csv(file, names = None, encoding='CP949')
rawdoc = rawdata['응답값']

In [265]:
rawdoc1 = [" ".join(re.findall('([가-힣]+)', row))+"." for row in rawdoc]
rawdoc1
with open("train.txt", 'w') as f:
    for s in rawdoc1:
        f.write(str(s) + '\n')

# ------------------

In [113]:
path = 'codedata/komoran_code_decode/clust_komoran_all.csv'
maxlen=100

In [8]:
path = 'codedata/clusterdata.csv'
maxlen=30

In [9]:
data = pd.read_csv(path, names = None)
clusternum=data['ClusterNo']
doc = data['rawdoc']
groupby_cluster = data['rawdoc'].groupby([data['ClusterNo']])
cluster_doc=groupby_cluster.apply(lambda x: x.tolist()).to_dict()

In [10]:
cluster_doc

{0: ['시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원.',
  '시원해서.',
  '시원해서.',
  '시원하다.',
  '시원하다.',
  '시원하다.',
  '시원해서.',
  '시원.',
  '시원하다.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원하다.',
  '시원함.',
  '시원해요.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원해서.',
  '시원해서.',
  '시원하다.',
  '시원함.',
  '시원함.'],
 1: ['맛이 깔끔하다.',
  '맛이 깔끔함.',
  '맛이 깔끔함.',
  '깔끔한 맛.',
  '맛이 깔끔하다.',
  '맛이 깔끔해서.',
  '맛이 깔끔함.',
  '맛이 깔끔함.',
  '맛이 깔끔하다.',
  '깔끔한 맛.',
  '깔끔한 맛.',
  '깔끔한 맛.',
  '깔끔한 

In [11]:
vocab = word2idx
def doc_to_line(cluster_doc, vocab):

    dropped = 0
    linecount = 0
    lines = []
    for line in cluster_doc:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
#         print(len(words))
        
        if len(words) > maxlen:
            dropped += 1
            continue
            
        words = ['<sos>'] + words
        words += ['<eos>']
        # vectorize
       
        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]
#         print(indices)
        lines.append(indices)
        
    return lines

In [12]:
def make_code(lines, vocab):
    eval_batch_size = 1
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    ntokens = len(vocab)
    
    
    code = []
    for i, batch in enumerate(test_data):
            source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
            indices = Variable(source)
            # output: batch x seq_len x ntokens
            output = autoencoder.encode(indices, lengths, noise=False)
#             print(output)
            code.append(output)
    return code , indices, lengths

In [48]:
doc_to_line(cluster_doc[409],vocab)

45
77


[]

In [12]:
cluster_title=[]
code_vec=[]
for c in cluster_doc.keys() : 
    lines= doc_to_line(cluster_doc[c], vocab)
    
    eval_batch_size = len(lines)
    
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    
    ntokens = len(vocab)
    for i, batch in enumerate(test_data):
        
        source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
        indices = Variable(source)
            # output: batch x seq_len x ntokens
        output = autoencoder.encode(indices, lengths, noise=False)
        
    code_vec.extend(output.data.numpy().tolist())
    
    batch_size, maxlen2=indices.size()
#     code_mean=torch.mean(code_cat,0).view(1,-1) #code 평균
#     print(lengths)
    decode_out = autoencoder.decode_mean(output, batch_size=batch_size ,maxlen = maxlen2,indices=indices, lengths=lengths)

    max_values, max_indices = torch.max(decode_out, 2)
    
    max_indices = max_indices.view(decode_out.size(0), -1).data.cpu().numpy().squeeze()
#     print(max_indices)


    # autoencoder output sentence
    words = [vocab2[x] for x in max_indices.squeeze()]
    # truncate sentences to first occurrence of <eos>
    truncated_sent = []
    for w in words:
        if w != '<eos>':
            truncated_sent.append(w)
        else:
            break
    chars = " ".join(truncated_sent)
            
    cluster_title.append(chars)
#     words = [vocab2[x] for x in max_indices]
# #     print(words)
#     # truncate sentences to first occurrence of <eos>
#     truncated_sent = []
#     for w in words:
#         if w != '<eos>':
#             truncated_sent.append(w)
#         else:
#             break
#     sent = " ".join(truncated_sent)
#     cluster_title.append(sent)

In [13]:
resultdoc={}
for i in range(len(cluster_title)):
    resultdoc[i]=cluster_title[i]

In [14]:
resultdoc

{0: '상큼해서.',
 1: '맛이 강하다.',
 2: '순한맛.',
 3: '맛있습니다.',
 4: '목넘김이 좋고 순해서.',
 5: '톡 쏘는 맛이 상쾌함.',
 6: '깔끔한맛.',
 7: '상큼한 순해서.',
 8: '입맛에 입맛에 맞음.',
 9: '맛이 상큼해서.',
 10: '맛',
 11: '목넘김이 좋고',
 12: '맛이 톡쏘는 순함.',
 13: '그냥 친근하다.',
 14: '뒷맛이 개운하다.',
 15: '깔끔하고 깔끔하고 개운하다.',
 16: '맛이 부드럽고 개운하다.',
 17: '맥주 맛이 괜찮음.',
 18: '목넘김이 좋고 개운하다.',
 19: '깔끔하고 깔끔하고 개운하다.',
 20: '부드럽고 고소하다.',
 21: '독특한 진하고 거품의',
 22: '청량감 흑맥주 청량감.',
 23: '맛과 좋고 좋고',
 24: '순한 부드럽고 개운하다.',
 25: '상큼한 상큼한 인지도.',
 26: '깨끗하고 아주 산뜻함.',
 27: '시원한 괜찮음.',
 28: '가장 가장 보편적.',
 29: '톡 쏘는 쏘는 좋으며 풍부하다.',
 30: '깊은 깊은 진한 강하다.',
 31: '보리 부드럽고 좋으며 상쾌하다.',
 32: '특유의 고소한 좋으며 상쾌함.',
 33: '고급스럽고 부드러운 디자인.',
 34: '가격이 저렴하고 가격대비 저렴한 잘맞습니다.',
 35: '내 입맛에 가장 맞고 맞습니다.',
 36: '호가든 좋아하는 좋아하는 파란색이라서.',
 37: '자주 마시는 먹는 거라서.',
 38: '시원한 느낌이 가벼움.',
 39: '상큼하고 맛있고 다양한맛이 산뜻함.',
 40: '부드러움 좋고 거품의 자극적이지 덜함.',
 41: '시원한 맛 때문임.',
 42: '대중적이고 쉽게 접할 있고 있고 익숙해졌다.',
 43: '칼로리가 낮고 없고 살수 덜하다.',
 44: '제일 제일 제일 괜찮음.',
 45: '부드러우면 맥주보다 없고 맥주들보다 것 생각이 마음에',
 46: '뭔가 좋고 좋으며 쓰지않음.

In [15]:
df = pd.DataFrame([resultdoc,cluster_doc]).T
df['clusterNo'] = df.index

In [16]:
df

,0,1,clusterNo
0,상큼해서.,"[시원해서., 시원함., 시원하다., 시원함., 시원해서., 시원함., 시원하다.,...",0
1,맛이 강하다.,"[맛이 깔끔하다., 맛이 깔끔함., 맛이 깔끔함., 깔끔한 맛., 맛이 깔끔하다.,...",1
2,순한맛.,"[부드럽다., 부드러움., 부드러움., 부드러움., 부드러움., 부드럽다., 부드러...",2
3,맛있습니다.,"[맛있다., 맛있음., 맛있어서., 맛있어서요., 맛있어요., 맛있다., 맛있어서....",3
4,목넘김이 좋고 순해서.,"[부드러운 목넘김이 좋다 맛도 부드러워 좋다., 목넘김이 좋고 부드럽다., 부드럽고...",4
5,톡 쏘는 맛이 상쾌함.,"[톡 쏘는 맛., 톡 쏘아서., 맛이 좋음 톡쏨., 톡 쏘는 맛이 좋다., 톡 쏘는...",5
6,깔끔한맛.,"[깔끔하다., 깔끔해요., 깔끔함., 깔끔함., 깔끔해서., 깔끔해서., 깔끔하므로...",6
7,상큼한 순해서.,"[부드러운 맛., 맛이 부드러워서., 부드러운 맛., 맛이 부드럽다., 맛이 부드러...",7
8,입맛에 입맛에 맞음.,"[맛이 입맛에 맞아서., 내 입맛에 맞아서., 내 입맛에 맞음., 맛이 입맛에 맞아...",8
9,맛이 상큼해서.,"[맛이 좋다., 맛이 좋아서., 맛이 좋아서., 맛이 좋아서., 맛이 좋아서., 맛...",9


In [17]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],row[0],  nn]) 
                         for nn in row[1]], axis=1)

In [18]:
df_new = pd.DataFrame(rows, columns=['clusterNo','title','raw_docs'])
df_new

,clusterNo,title,raw_docs
0,0,상큼해서.,시원해서.
1,0,상큼해서.,시원함.
2,0,상큼해서.,시원하다.
3,0,상큼해서.,시원함.
4,0,상큼해서.,시원해서.
5,0,상큼해서.,시원함.
6,0,상큼해서.,시원하다.
7,0,상큼해서.,시원함.
8,0,상큼해서.,시원해서.
9,0,상큼해서.,시원함.


In [19]:
df_new.to_csv('1109_title_result_autoencoder_decode_raw.csv',sep=',',encoding='CP949')

# ------------------------------------------------

# -------------------codevec to csv

In [116]:
cluster_title=[]
code_vec=[]
for c in cluster_doc.keys() : 
    lines= doc_to_line(cluster_doc[c], vocab)
    
    eval_batch_size = len(lines)
    
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    
    ntokens = len(vocab)
    for i, batch in enumerate(test_data):
        
        source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
        indices = Variable(source)
            # output: batch x seq_len x ntokens
        output = autoencoder.encode(indices, lengths, noise=False)
        
    code_vec.extend(output.data.numpy().tolist())

In [119]:
df= pd.DataFrame(code_vec)

In [120]:
df1 = pd.concat([clusternum,doc,df], axis=1)
df1.to_csv('cluster_code_vec_komoran_all.csv', index=False)

# ------------------------------------------------codevec to csv end